In [ ]:
import sys, json
from itertools import groupby
sys.path.append(r"../")

from PIL import Image, ImageDraw
from pathlib import Path
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

from detectron2.data import MetadataCatalog
import cameratransform as ct

import ct_assist as cta
from ct_assist import reference_detection as rd
from ct_assist import transform as ctf
from ct_assist.utils import accuracy

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_dir = r"../../dataset2"
anno_dir = data_dir + "/annotations"
im_dir = Path(r"D:\University\2020-2021\Internship\dataset2\images")


In [ ]:
with open(r"D:\University\2020-2021\Internship\dataset2\annotations\simulation_properties.json") as f:
    sims = json.load(f)
sim = sims[0]  # Only one simulation filed
sim.keys()

In [ ]:
with open(r"D:\University\2020-2021\Internship\dataset2\annotations\images.json") as f:
    imgs = json.load(f)

In [ ]:
def load():
    with open(anno_dir + "/images.json") as f:
        img_lst = json.load(f)

    with open(anno_dir + "/instance_segmentation.json") as f:
        anno_lst = sorted(json.load(f), key=lambda x: x["image_id"])

    predictor, cfg = rd.load_model(return_cfg=True)

    X = []
    Y_test = []
    X_area = []
    X_cprops = []
    Y_area = []

    classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get("thing_classes")
    a = 0
    for k, g in groupby(tqdm(anno_lst), lambda x: x["image_id"]):
        a += 1
        record = {"height":1080, "width":1920}
        record["image_id"] = k
        record["file_name"] = im_dir / img_lst[k].get("file_name")
        img = Image.open(record["file_name"])
        arr = np.asarray(img)[...,:3]

        pred = predictor(arr)
        inst_dict = rd.instances_to_dict(pred, classes)

        ref = rd.extract_reference(inst_dict, always_warn=False)
        if len(ref) == 0:
            continue
        if len(ref[0][0]) == 0:
            continue
        X.append({"img": img, "reference": [], "height": [], "STD": [], "meta_data": {"focal_length": 50, "image_size": (1920, 1080), "sensor_size": (36, 24)}, "multi": True})
        Y_test.append([img_lst[k].get(key) for key in ["roll_deg", "tilt_deg", "heading_deg", "elevation"]])
        for r, h, s in ref:
            X[-1]["reference"].append(r)
            X[-1]["height"].append(h)
            X[-1]["STD"].append(s)

        sim_id = img_lst[k].get("simulation_id")
        if sim_id == 0.0:
            segs = None
            for anno in filter(lambda x: x["category_id"] == 0, g):                
                segs = anno["segmentation"]

            if segs:
                frame = img_lst[k].get("frame")
                X_area.append(X[-1].copy())
                X_area[-1]["image_coords"] = [np.array(list(zip(a[::2], a[1::2]))) for a in segs]
                Y_area.append(sim["area_cm2_per_frame"][k] / 10_000) 
                X_cprops.append((Y_test[-1], X[-1]["meta_data"], X_area[-1]["image_coords"]))
    
    return X, Y_test, X_area, X_cprops, Y_area

X, Y_test, X_area, X_cprops, Y_area = load()

In [ ]:
X_c = X.copy()
Y_c = Y_test.copy()

In [ ]:
# Test area with known camera properties
def make_cam(roll_deg, tilt_deg, heading_deg, elevation, focal_length, image_size, sensor_size, img_cords):

    proj = ct.RectilinearProjection(focallength_mm=focal_length,
                                         sensor=sensor_size,
                                         image=image_size)
    # initialize the camera
    cam = ct.Camera(proj, ct.SpatialOrientation(elevation_m=elevation,
                                          tilt_deg=tilt_deg,
                                          heading_deg=heading_deg,
                                          roll_deg=roll_deg
                                          ))
    
    
    points = (cam.spaceFromImage(points=x) for x in img_cords)
    areas = sum(accuracy.calc_area(poly[:, :2]) for poly in points)
    return areas
areas = [make_cam(*ext, **inte, img_cords=im) for ext, inte, im in tqdm(X_cprops)]
print(areas)
mean_squared_error(Y_area, areas)

In [ ]:
score2, pred_area = accuracy.area(X_area, Y_area)
score2

In [ ]:
score, ypred, ytue = accuracy.camera_properties(X_c, Y_c)

In [ ]:
score

In [ ]:
fig, ax = plt.subplots(5, figsize=(15, 15))

# for i in [0, 1, 3]:
# ax[0].plot(ypred[...,0], label="pred")
# ax[0].plot(ytue[..., 0], label="true")

ax[0].scatter(range(ytue.shape[0]), ytue[..., 0], label="True")
ax[0].scatter(range(ytue.shape[0]), ypred[...,0], label="Predicted")
ax[0].set_title("Roll degree")


# ax[1].plot(ypred[...,1], label="pred")
# ax[1].plot(ytue[..., 1], label="true")
ax[1].scatter(range(ytue.shape[0]), ytue[..., 1], label="True")
ax[1].scatter(range(ytue.shape[0]), ypred[...,1], label="Predicted")
ax[1].set_title("Tilt degree")

# ax[2].plot(ypred[...,3], label="pred")
# ax[2].plot(ytue[..., 3], label="true")
ax[2].scatter(range(ytue.shape[0]), ytue[..., 3], label="True")
ax[2].scatter(range(ytue.shape[0]), ypred[...,3], label="Predicted")
ax[2].set_title("Elevation (m)")

ax[3].set_yscale('log')
ax[3].scatter(range(len(X_area)), Y_area, label="True")
ax[3].scatter(range(len(X_area)), pred_area, label="Predicted")
ax[3].scatter(range(len(X_area)), areas, label="Predicted with known camera properties")
ax[3].set_title("Area (m$^{2}$) (log-scaled)")


# ax[3].set_yscale('log')
ax[4].scatter(range(len(X_area)), Y_area, label="True")
ax[4].scatter(range(len(X_area)), pred_area, label="Predicted")
ax[4].scatter(range(len(X_area)), areas, label="Predicted with known camera properties")
ax[4].set_title("Area (m$^{2}$)")



plt.legend()
plt.show()
    